In [ ]:

'''
    Naman Deep Singh, Backtracking line search
    for Gradient descent on a one layer-linear classifier for MNIST data.
'''


import numpy as np
import numpy as np
import scipy.io as sio





def backprop(X, y, W, lamb, n_classes, batch_size):
    F_x = np.dot(X,W)

    C = np.max(F_x, axis=1, keepdims=True)
    exp_scores = np.exp(F_x - C)

    S = exp_scores/np.sum(exp_scores, axis=1, keepdims=True)

    #  log-loss
    log_S = -np.log(S[np.arange(batch_size), y])
    L = np.sum(log_S)/batch_size

    # Add regularization using the L2 norm
    L_reg = 0.5*lamb*np.sum(W*W)
    L+= L_reg

    # Gradient of the loss with respect to scores
    grad = S.copy()
    # Substract 1 from the scores of the correct class
    grad[np.arange(batch_size),y] -= 1
    grad /= batch_size

    # Gradient of the loss with respect to weights
    grad_W = X.T.dot(grad)

    # Add gradient regularization
    grad_W+= lamb*W

    return L, grad_W

                                #########     Function for backtracking rule        ############
######   X = train data, y= train labels    W = weights
def backtrack(W, grad_W, loss, alpha, X, y, lamb, t, beta, n_classes, batch_size):
    count = 1
    loss_val,_ = backprop(X, y, W-t*grad_W, lamb, n_classes, batch_size)    #####     Function value at x-t*x'
    while loss_val <= loss-(t*alpha)*np.square(np.linalg.norm(grad_W, 'fro')):              ########      f(x-tx') > f(x) - t*alpha*|x'|^2
        t *= beta
        loss_val,_ = backprop(X, y, W-t*grad_W, lamb, n_classes, batch_size)

    return t

grad_W = np.zeros_like(W)
                    #################      Condition either full run of training data or ||grad_W||<1e-3      ##########
def grad_descent(X, y, lamb, grad_W, W, n_classes):
    # W = np.zeros(shape=(784, n_classes))
    batch_size = 100
    lamb = 1e-2   ### lambda in weight regularizer
    t = 1
    alpha = 0.4
    beta = 0.6      ###    For backtracking
    while np.linalg.norm(grad_W, 'fro') < 1e-3:
        start = 0
        while start < number_of_samples-batch_size :
            loss, grad_W = backprop(X[start:start+batch_size,:], y[start:start+batch_size], W, lamb, n_classes, batch_size)
            alpha = backtrack(W, grad_W, loss, alpha, X[start:start+batch_size,:], y[start:start+batch_size], lamb, t, beta, n_classes, batch_size)
            # count+=1
            t = alpha
            W = W - alpha*grad_W
            start+=batch_size
    return W

Weights = grad_descent(X, y, grad_W, W)